<a href="https://colab.research.google.com/github/jaya-shankar/education-impact/blob/master/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf education-impact
!rm education-impact

rm: cannot remove 'education-impact': No such file or directory


In [1]:
!git clone https://github.com/jaya-shankar/education-impact.git
!pip install tensorflow_decision_forests

Cloning into 'education-impact'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 126 (delta 59), reused 70 (delta 22), pack-reused 0
Receiving objects: 100% (126/126), 990.64 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
root = "education-impact/" 
datasets_path = {
                    "infant_mortality"              :  root+ "datasets/Infant_Mortality_Rate.csv",
                    "child_mortality"               :  root+ "datasets/child_mortality_0_5_year_olds_dying_per_1000_born.csv",
                    "children_per_woman"            :  root+ "datasets/children_per_woman_total_fertility.csv",
                    "co2_emissions"                 :  root+"datasets/co2_emissions_tonnes_per_person.csv",
                    "population"                    :  root+ "datasets/converted_pop.csv",
                    "food_supply"                   :  root+ "datasets/food_supply_kilocalories_per_person_and_day.csv",
                    "gdp_per_captia"                :  root+ "datasets/gdp_per_capita_yearly_growth.csv",
                    "gini_index"                    :  root+ "datasets/gini.csv",
                    "life_expectancy"               :  root+ "datasets/life_expectancy_years.csv",
                    "malnutrition"                  :  root+ "datasets/malnutrition_weight_for_age_percent_of_children_under_5.csv",
                    "poverty_index"                 :  root+ "datasets/mincpcap_cppp.csv",
                    "maternal_mortality"            :  root+ "datasets/mmr_who.csv",
                    "people_in_poverty"             :  root+ "datasets/number_of_people_in_poverty.csv",
                    "primary_completion"            :  root+ "datasets/primary_school_completion_percent_of_girls.csv",
                    "ratio_b/g_in_primary"          :  root+ "datasets/ratio_of_girls_to_boys_in_primary_and_secondary_education_perc.csv",
                    "wcde-15--24"                   :  root+ "datasets/wcde-15--24.csv",
                    "wcde-25--34"                   :  root+ "datasets/wcde-25--34.csv",
                    "wcde-35--44"                   :  root+ "datasets/wcde-35--44.csv",
                    "wcde-45--54"                   :  root+ "datasets/wcde-45--54.csv",
                    "wcde-55--64"                   :  root+ "datasets/wcde-55--64.csv",
                    "wcde-65--74"                   :  root+ "datasets/wcde-65--74.csv",
                    "wcde-75--84"                   :  root+ "datasets/wcde-75--84.csv",
                    "wcde-85--94"                   :  root+ "datasets/wcde-85--94.csv",
                    "wcde-95--"                     :  root+ "datasets/wcde-95--.csv",
                 
                }

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_decision_forests as tfdf

In [5]:
countries_arr = []
for path in datasets_path:
  df = pd.read_csv(datasets_path[path])
  print(f"{'Factor: ' + path:<30} count: {len(set(df.Country.unique()))}")
  

Factor: infant_mortality       count: 266
Factor: child_mortality        count: 197
Factor: children_per_woman     count: 202
Factor: co2_emissions          count: 194
Factor: population             count: 197
Factor: food_supply            count: 179
Factor: gdp_per_captia         count: 221
Factor: gini_index             count: 195
Factor: life_expectancy        count: 195
Factor: malnutrition           count: 156
Factor: poverty_index          count: 195
Factor: maternal_mortality     count: 184
Factor: people_in_poverty      count: 145
Factor: primary_completion     count: 195
Factor: ratio_b/g_in_primary   count: 200
Factor: wcde-15--24            count: 202
Factor: wcde-25--34            count: 202
Factor: wcde-35--44            count: 202
Factor: wcde-45--54            count: 202
Factor: wcde-55--64            count: 202
Factor: wcde-65--74            count: 202
Factor: wcde-75--84            count: 202
Factor: wcde-85--94            count: 202
Factor: wcde-95--              cou

from the above output
- **malnutrition & people in povery** have least no of countries
- **infant mortality & gdp per captia** have highest no of countries

*Doubt:* Does having more data for one factor will make the decision tree bias?


###Steps
1. create a csv file such that each row contains all values of particular year & country present
2. the output for each row is year + 40 years corresponding value 
    1. **outputs** - life expectany, education level, gdp




In [6]:
PREDICT_FUTURE  = 40
OUTPUTS         = ['life_expectancy', 'gdp_per_captia', 'primary_completion' ]

In [8]:

countries = list(pd.read_csv('education-impact/datasets/Infant_Mortality_Rate.csv').Country.unique())
years     = [y for y in range(1960,2015-PREDICT_FUTURE+1)]

In [9]:
keys=[]
for y in years:
  for c in countries:
    keys.append((c,str(y)))

In [10]:
big_dic = {k : [] for k in keys}
for path in datasets_path:
  df = pd.read_csv(datasets_path[path])
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][k[1]])
    except:
      big_dic[k].append(np.NaN)
 

In [ ]:
print(len(datasets_path))

24


In [11]:
for output_path in OUTPUTS:
  df = pd.read_csv(datasets_path[output_path])
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][str(int(k[1])+PREDICT_FUTURE)])
    except:
      big_dic[k].append(np.NaN)


In [12]:
columns = [k for k in datasets_path ]
output_columns = ["o_"+o for o in OUTPUTS]
columns.extend(output_columns)

In [17]:
input_df = pd.DataFrame.from_dict(big_dic,orient='index', columns = columns)
output_df = input_df[["o_"+o for o in OUTPUTS]]
input_df.drop(labels=["o_"+o for o in OUTPUTS], axis = 1, inplace=True)

,infant_mortality,child_mortality,children_per_woman,co2_emissions,population,food_supply,gdp_per_captia,gini_index,life_expectancy,malnutrition,poverty_index,maternal_mortality,people_in_poverty,primary_completion,ratio_b/g_in_primary,wcde-15--24,wcde-25--34,wcde-35--44,wcde-45--54,wcde-55--64,wcde-65--74,wcde-75--84,wcde-85--94,wcde-95--
"(Aruba, 1960)",NaN,NaN,4.82,NaN,NaN,NaN,2.450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.64,9.12,7.18,5.51,4.24,3.41,2.75,2.40,2.10
"(Africa Eastern and Southern, 1960)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(Afghanistan, 1960)",NaN,357.0,7.45,0.046,9000000.0,NaN,1.430,34.4,45.0,NaN,2.08,NaN,NaN,NaN,NaN,1.12,0.82,0.54,0.34,0.23,0.16,0.11,0.08,2.10
"(Africa Western and Central, 1960)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(Angola, 1960)",NaN,308.0,7.48,0.101,5450000.0,NaN,2.260,55.8,46.3,NaN,7.53,NaN,NaN,NaN,NaN,1.50,1.09,0.73,0.47,0.31,0.20,0.14,0.09,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(Kosovo, 1975)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(Yemen, Rep., 1975)",180.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(South Africa, 1975)",85.7,121.0,5.29,7.350,25200000.0,2900.0,-0.651,67.3,58.3,NaN,8.46,NaN,NaN,NaN,NaN,13.98,12.89,11.18,9.14,7.18,5.42,3.98,2.84,2.02
"(Zambia, 1975)",96.1,156.0,7.36,0.825,4940000.0,2320.0,-3.920,69.0,55.1,NaN,4.02,NaN,NaN,56.7,NaN,11.84,10.52,7.31,4.76,2.93,1.82,1.34,0.94,0.61


From above output
- if we dont drop any rows our table size = 4256 entries
- if we drop rows containing any if all of outputs missing then our table size = 3039 entries
- if we drop rows containing any one of output missing then our table size = 1745 entries

so, I think its is better to go with second choice and build different models, but not sure it will not effect performance of the model


now we have the dataframe containing both inputs and ouputs,our next step is
1. split the data into train & test data
  1. try to split data based on continents to reduce bias
2. build DF model using tensorflow
3. check the accuracy of the model

In [23]:
X_train, X_test, y_train, y_test = train_test_split(input_df, output_df, test_size=0.50, random_state=43)

In [31]:
frames      = [X_train,y_train['o_life_expectancy']]
le_model_df = pd.concat(frames,axis=1)
le_model_df.dropna(subset=['o_life_expectancy'],inplace=True)
le_model_df

,infant_mortality,child_mortality,children_per_woman,co2_emissions,population,food_supply,gdp_per_captia,gini_index,life_expectancy,malnutrition,poverty_index,maternal_mortality,people_in_poverty,primary_completion,ratio_b/g_in_primary,wcde-15--24,wcde-25--34,wcde-35--44,wcde-45--54,wcde-55--64,wcde-65--74,wcde-75--84,wcde-85--94,wcde-95--,o_life_expectancy
"(Papua New Guinea, 1960)",132.8,198.0,6.28,0.0796,2260000.0,NaN,1.450,52.6,53.7,NaN,1.09,NaN,NaN,NaN,NaN,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,63.1
"(Malta, 1969)",26.0,29.2,2.02,2.0300,321000.0,3140.0,6.060,28.0,70.6,NaN,5.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.8
"(Greece, 1975)",23.0,26.3,2.50,4.2900,9010000.0,3340.0,5.210,40.2,74.6,NaN,16.10,NaN,NaN,95.2,0.909,17.98,17.6,14.0,11.62,10.0,8.18,7.16,5.83,4.82,80.9
"(Ireland, 1966)",23.4,27.1,3.90,5.3300,2830000.0,3390.0,0.574,38.9,70.7,NaN,10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.4
"(Vanuatu, 1971)",74.4,104.0,6.20,0.6670,88000.0,2650.0,1.190,37.6,57.3,NaN,3.63,NaN,NaN,NaN,0.913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(Senegal, 1974)",115.6,261.0,7.30,0.3860,4800000.0,2260.0,1.090,54.7,49.4,NaN,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.8
"(St. Kitts and Nevis, 1962)",NaN,114.0,NaN,NaN,51000.0,1780.0,1.200,40.0,60.0,NaN,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.7
"(Cote d'Ivoire, 1961)",206.7,309.0,7.43,0.1520,3630000.0,2330.0,9.530,47.7,46.1,NaN,6.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.5
"(Nicaragua, 1968)",120.2,180.0,6.94,0.5440,2260000.0,2130.0,-1.810,49.2,56.5,NaN,8.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0


In [33]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(le_model_df, label="o_life_expectancy",max_num_classes=500)

In [34]:
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

23/23 [==============================] - 5s 3ms/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [35]:
frames      = [X_test,y_test['o_life_expectancy']]
le_model_test_df = pd.concat(frames,axis=1)
le_model_test_df.dropna(subset=['o_life_expectancy'],inplace=True)
le_model_test_df

,infant_mortality,child_mortality,children_per_woman,co2_emissions,population,food_supply,gdp_per_captia,gini_index,life_expectancy,malnutrition,poverty_index,maternal_mortality,people_in_poverty,primary_completion,ratio_b/g_in_primary,wcde-15--24,wcde-25--34,wcde-35--44,wcde-45--54,wcde-55--64,wcde-65--74,wcde-75--84,wcde-85--94,wcde-95--,o_life_expectancy
"(Ukraine, 1971)",27.9,34.2,2.06,9.7900,47400000.0,NaN,1.650,31.3,71.0,NaN,6.240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.5
"(St. Kitts and Nevis, 1960)",NaN,127.0,NaN,NaN,51200.0,NaN,1.200,40.0,58.3,NaN,2.570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.9
"(Portugal, 1967)",61.5,77.6,3.13,1.4200,8750000.0,2950.0,7.620,45.8,66.3,NaN,8.240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.4
"(Mauritania, 1971)",103.5,191.0,6.77,0.3380,1180000.0,1960.0,-0.680,40.3,55.2,NaN,2.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.4
"(Uganda, 1960)",131.4,223.0,7.00,0.0623,6770000.0,NaN,1.840,53.5,49.0,NaN,3.140,NaN,NaN,NaN,NaN,5.78,4.23,3.22,2.84,1.39,0.42,0.12,0.03,0.01,49.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(Philippines, 1961)",64.6,100.0,7.09,0.3210,27200000.0,1790.0,2.450,44.8,61.5,NaN,2.630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.2
"(Palau, 1972)",NaN,47.4,NaN,NaN,12300.0,NaN,10.100,40.0,59.4,NaN,30.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0
"(Burkina Faso, 1969)",145.0,318.0,6.59,0.0219,5530000.0,1600.0,0.823,48.3,43.6,NaN,0.934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.6
"(Vanuatu, 1972)",71.7,99.9,6.13,0.6870,90800.0,2610.0,1.190,37.6,57.7,NaN,3.680,NaN,NaN,NaN,0.917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.7


In [36]:
# Convert it to a TensorFlow dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(le_model_test_df, label="o_life_expectancy", max_num_classes=500)

# Evaluate the model
model.compile(metrics=["accuracy"])
print(model.evaluate(test_ds))

23/23 [==============================] - 1s 12ms/step - loss: 0.0000e+00 - accuracy: 0.0516
[0.0, 0.05163991451263428]


In [37]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)